In [10]:
from IPython.display import IFrame

import os
import numpy as np
import pandas as pd
import pickle

import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [4]:
def display_artist(track_info):
## create a single string with all artist names
    return " / ".join([x['name'] for x in track_info['artists']])

In [5]:
def create_artist_input_prompt(artist_names):
    user_prompt = ""
    valid_input = []
    
    for i in range(len(artist_names)):
        my_num = i
        user_prompt += f"Enter {my_num} for '{artist_names[i]}'\n"
        valid_input.append(str(my_num))
    ## space is always none of the above
    user_prompt += "Enter [SPACE] for 'None of the above'\n"
    valid_input.append(" ")
    
    return valid_input, user_prompt

In [6]:
def recommend_hot_song(hot_songs_df, not_this_title):
    myrec = hot_songs_df.sample()
    
    if (myrec.title == not_this_title).all():
        print("DEBUG: try again")
        myrec = hot_songs_df.sample()
    return myrec

In [8]:
## Load hot songs df
infile_list = ['Data/db/hot/billboard_hot100_2021-02-14.csv', 
               'Data/db/hot/uk_hot100_2021-02-14.csv', 
               'Data/db/hot/hot_songs_miri.csv']

dfs = []
for fname in infile_list:
    dfs.append(pd.read_csv(fname, index_col=None))
    
hot_songs_df = pd.concat(dfs, axis=0)
hot_songs_df = hot_songs_df.drop_duplicates().reset_index(drop=True)

In [11]:
## load general songs df --- NEED TO FIX NULL VALUES IN INPUT FILES FOR THIS TO WORK 
# indir = 'Data/db'
# infile_list = os.listdir(indir)

# dfs = []
# for fname in infile_list:
#     if fname.endswith('.csv'):
#         dfs.append(pd.read_csv(os.path.join(indir, fname), index_col=None))
    
# gen_songs_df = pd.concat(dfs, axis=0)
# gen_songs_df = gen_songs_df.drop_duplicates().reset_index(drop=True)

In [26]:
## load general songs df -- keep using my old data for now
gen_songs_df = pd.read_csv('Data/db/day4/nina_day4_labeled_tracks.csv')
if gen_songs_df[gen_songs_df.isnull().any(axis=1)].shape[0] != 0:
    print("Error! General songs DB has records with null values")
gen_songs_df = gen_songs_df.drop_duplicates().reset_index(drop=True)

In [27]:
gen_songs_df.shape

(2377, 5)

In [28]:
## define constants
MODEL_FEATURES = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
        'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

LABELS_COLNAME = 'k7_cluster_label'

In [29]:
## create cluster membership filters
clusters = np.unique(gen_songs_df[LABELS_COLNAME])
cluster_members = {}
for cluster in clusters:
    cluster_members[cluster] = np.where(gen_songs_df[LABELS_COLNAME] == cluster)

In [30]:
## get models
fitted_scaler = load("Model/nina-scaler.pickle")
fitted_kmeans = load("Model/nina-kmeans_7.pickle")

In [ ]:
## ---- ready for user query ----
userinput_songname = input("enter song name: ")

results = sp.search(q=userinput_songname, type='track', limit=1, market="GB")

if results is None:
    print('Sorry, that track name is not recognised')
else:
    # collect track details
    track_info = results['tracks']['items'][0]
    
    print(f"Based on the track ({track_info['id']}):\n\t{track_info['name']}\n\tby {display_artist(track_info)}\n")
    
    IFrame(src="https://open.spotify.com/embed/track/"+track_info['id'],
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )
    
    if track_info['id'] in hot_songs_df.id.to_list():
        randhot = recommend_hot_song(hot_songs_df, selected_song.title)
        print(f"I recommend HOT song: {randhot.title.values[0]} by {randhot.pp_artist.values[0]}")
        IFrame(src="https://open.spotify.com/embed/track/"+randhot.id.values[0],
                   width="320",
                   height="80",
                   frameborder="0",
                   allowtransparency="true",
                   allow="encrypted-media",
                  )
    else:
        # recommend something similar
        ## is this track already in our database?
        if track_info['id'] in gen_songs_df.id:
            print('I know this one!')
            track_label = gen_songs_df[gen_songs_df.id == track_info.id][CLUSTERING]
        else:
            ## get the label for these audio features
            full_track_audio_features = sp.audio_features(track_info['id'])[0]
            if full_track_audio_features is None:
                print("Error! I don't know what that song sounds like :(")
            else:
                track_audio_features = pd.DataFrame({ky: full_track_audio_features[ky] for ky in MODEL_FEATURES},
                                                    index=[0])

                track_audio_features_scaled = fitted_scaler.transform(track_audio_features)
                track_audio_features_scaled_df = pd.DataFrame(track_audio_features_scaled, 
                                                              columns = track_audio_features.columns)
                track_label = fitted_kmeans.predict(track_audio_features_scaled_df)[0]

        randsimilar = gen_songs_df.loc[cluster_members[track_label]].sample()
        print(f"I recommend SIMILAR song (cluster {track_label}):")
        print(f"\t{randsimilar.title.values[0]}\n\tby {randsimilar.pp_artist.values[0]}")
        IFrame(src="https://open.spotify.com/embed/track/"+randsimilar.id.values[0],
                   width="320",
                   height="80",
                   frameborder="0",
                   allowtransparency="true",
                   allow="encrypted-media",
                  )